# Training model

In [8]:
#Import Packages
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns

df = pd.read_csv('Boliga - Final for training.csv')

In [30]:
#Split into X and Y 
df_dum = pd.get_dummies(df, drop_first = True, columns = ['isForeclosure', 'Type'])
X = df_dum.loc[:, df_dum.columns != ['price', 'Kommune']
y = df_dum['price']

In [31]:
X

,basementSize,buildYear,Ejerudgift,lotSize,rooms,size,Kommune,Relativ Ledighed,Grundskole,Gymnasiale uddannelser,Erhvervsfaglige uddannelser,KVU,MVU,Bacheloruddannelser,LVU,Kommunal_gennemsnitsinkomst_2017,Total_reported,Population_in_urban_development,Socioeconomic_index,expenses_per_school_student,average_class_size,expenses_sport_and_other_cultural_activities,lake_distance,forest_distance,doctor_distance,supermarket_distance,school_distance,daycare_distance,hospital_distance,train_distance,pharmacy_distance,library_distance,coast_distance,junction_distance,isForeclosure_True,Type_Ejerlejlighed,Type_Fritidsgrund,Type_Fritidshus,Type_Helårsgrund,Type_Landejendom,Type_Rækkehus,Type_Villa,Type_Villalejlighed
0,0,1767,16872,0,6.0,280,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.13204,7.22733,0.11111,0.24096,0.35005,0.39428,2.67332,1056.50,0.38665,0.92771,0.41379,4.65646,0,1,0,0,0,0,0,0,0
1,88,1880,5702,0,4.0,83,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.20441,7.30106,0.29860,0.59852,0.42607,0.43139,2.95715,1411.22,0.70159,1.23705,0.03508,4.94359,0,1,0,0,0,0,0,0,0
2,0,1755,9486,0,7.0,274,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.18899,7.36108,0.26852,0.71105,0.48610,0.40131,2.92707,1381.15,0.67141,1.29708,0.07032,4.91351,0,1,0,0,0,0,0,0,0
3,0,1880,17609,0,7.0,296,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.20441,7.30106,0.29860,0.59852,0.42607,0.43139,2.95715,1411.22,0.70159,1.23705,0.03508,4.94359,0,1,0,0,0,0,0,0,0
4,0,1873,5590,0,7.0,163,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.25292,7.23167,0.39571,0.64693,0.36048,0.52840,3.05426,1508.33,0.79860,1.16766,0.12353,5.04060,0,1,0,0,0,0,0,0,0
5,0,1873,5739,0,4.0,164,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.29041,7.15658,0.40142,0.73205,0.36620,0.53411,3.05997,1514.05,0.80431,1.09268,0.14719,5.04631,0,1,0,0,0,0,0,0,0
6,0,1870,4461,0,3.0,76,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.09059,6.95677,0.62901,0.81589,0.40758,0.76180,3.11336,1567.43,0.82495,0.89286,0.19007,5.09980,0,1,0,0,0,0,0,0,0
7,0,1872,9790,0,6.0,132,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.07550,6.94177,0.59185,0.68329,0.27518,0.70480,3.03906,1449.73,0.73779,0.87776,0.22029,5.02540,0,0,0,0,0,0,0,0,0
8,0,1900,5606,0,4.0,122,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.09335,6.95952,0.59336,0.77569,0.45853,0.72605,3.16441,1618.48,0.82770,0.89561,0.14033,5.15075,0,1,0,0,0,0,0,0,0
9,0,1796,2771,0,3.0,125,København,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.21384,6.74242,0.40809,0.26543,0.57113,0.53086,2.80540,1000.50,0.26553,0.80509,0.09079,4.85385,0,1,0,0,0,0,0,0,0


### Split data into Test, Development, Validation and Train

In [25]:
from sklearn.model_selection import train_test_split

X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size = 0.5, random_state = 1)

### Scale, fit and transform data

In [5]:
from sklearn.preprocessing import StandardScaler

stdscale = StandardScaler(with_mean = 1, with_std = True)

#Scale explanatory variables
X_train_s = stdscale.fit_transform(X_train)
X_test_s = stdscale.transform(X_test)


#Scale target variable (because of regression model - no need when categorical)
y_train_s = stdscale.fit_transform(y_train)
y_test_s = stdscale.transform(y_test)


In [ ]:
#Creates pipeline for Lasso-model:
pipe_lr = make_pipeline(PolynomialFeatures(include_bias=False, degree = 3), 
                        StandardScaler(with_mean = 0, with_std = True ),
                        Lasso())

### Optimize hyperparameter with K-fold CV 

In [ ]:
#Find optimized lambda
from sklearn.model_selection import KFold

lambdas =  np.logspace(-4, 4, 12) # define lambda - devide into 12 lambdas between 10**-4 and 10**4
kfolds = KFold(n_splits=5) # define nr. of folds

#Creates lists to append values of the loop:
mse_val_avr = []
mse_test_avr = []
lambda_list = []

#Outer-loop (Lambdas)
for lambda_ in lambdas:
#Inner-loop (Folds)
    mse_val_ = []
    mse_test_ = []
    #Split the dataset into 5 folds, and loop through each fold. 
    # In the iteration, the fold in question acts as the training-set. 
    for train_idx,val_idx in kfolds.split(X_dev, y_dev):
    # train model and compute MSE on test fold
        pipe_Lasso_CV = make_pipeline(PolynomialFeatures(include_bias=False, degree = 3), 
                        StandardScaler(with_mean = 0, with_std = True ),
                        Lasso(alpha = lambda_, random_state=1))
        # Assign X_train & y_train by extracting elements if the folds from X_dev and y_dev
        X_train, y_train = X_dev.iloc[train_idx], y_dev[train_idx]
        # Assign X_val & y_val by extracting elements of the folds from X_dev and y_dev
        X_val, y_val = X_dev.iloc[val_idx], y_dev[val_idx] 
        # Fit Lasso-model to training data
        pipe_Lasso_CV.fit(X_train, y_train)   
        #Calculate and append the mse into mse_val list:
        mse_val_.append(mse(pipe_Lasso_CV.predict(X_val), y_val))
        ##Calculate and append the mse into mse_test list:
        mse_test_.append(mse(pipe_Lasso_CV.predict(X_test), y_test))
        
    mse_val_avr.append(sum(mse_val_)/len(mse_val_))
    mse_test_avr.append(sum(mse_val_)/len(mse_val_))
    mse_val.append(mse_val_)
    mse_test.append(mse_test_)

df_val = pd.DataFrame(mse_val_avr, index = lambdas)
df_test = pd.DataFrame(mse_test_avr, index = lambdas)
df_join = pd.concat([df_test, df_val], axis=1)
df_with_index = df_join.reset_index()

#Convert to list
mses = df_with_index.values.tolist()
